In [0]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [5]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [6]:
import syft as sy

W0722 16:41:10.081966 140337387378560 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0722 16:41:10.101308 140337387378560 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [8]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

**Lesson : Basic Remote Execution in PySyft**

In [0]:
bob = sy.VirtualWorker(hook, id='bob')

In [10]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [13]:
bob._objects

{24457577156: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

24457577156

In [16]:
x.id

85843761568

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [19]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}